In [11]:
from keras.models import load_model
import cv2
import numpy as np
from skimage.transform import resize

# Cargar el modelo y el clasificador de rostros
modelo = load_model('CNN/modelPrueba111-013DEF.model')
face_clsfr = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

#Abrir el video o la cámara
source = cv2.VideoCapture('Deconocido.mp4')
#source = cv2.VideoCapture(0)
labels_dict = {0: 'jair', 1: 'Cris', 2: 'Messi', 3: 'Dylan', 4: 'Desconocido'}
color_dict = {0: (0, 255, 0), 1: (0, 0, 255), 2: (255, 0, 0), 3: (0, 255, 255), 4: (0, 0, 0)}

while True:
    ret, img = source.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_clsfr.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face_img = gray[y:y+w, x:x+w]
        resized = resize(face_img, (150, 150, 3))


        result = modelo.predict(np.expand_dims(resized, axis=0))

        label = np.argmax(result, axis=1)[0]

        cv2.putText(img, '{}'.format(label), (x, y-50), 1, 1.3, (86, 155, 35), 1, cv2.LINE_AA)

        if label in labels_dict:
            cv2.putText(img, '{}'.format(labels_dict[label]), (x, y-25), 2, 1.1, (255, 255, 0), 1, cv2.LINE_AA)
            cv2.putText(img, 'Estudiante', (x, y+h+30), 2, 0.8, (255, 255, 0), 1, cv2.LINE_AA)
            cv2.rectangle(img, (x, y), (x+w, y+h), color_dict[label], 2)
        else:
            cv2.putText(img, 'Desconocido', (x, y-25), 2, 1.1, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.rectangle(img, (x, y), (x+w, y+h), color_dict[4], 2)

    cv2.imshow('LIVE', img)
    key = cv2.waitKey(1)

    if key == 27:  # Presiona la tecla Esc para salir
        break

cv2.destroyAllWindows()
source.release()